# Judd Creek Project

In [ ]:
# This first cell just loads a few modules that we'll need.
# Run it by clicking on any of the text, holding down the shift key, and then pressing enter.
import os
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from judd import *

In [ ]:
# 1. First, let's open on of the downloaded data files from this site:
# https://green2.kingcounty.gov/hydrology/DataDownload.aspx

# 2. To select the right station, you'll need to use the map:
# https://green2.kingcounty.gov/hydrology/GaugeMap.aspx

# 3. Upload the data file you downloaded to the Jupyter Hub, in the data directory, and then change the
# following path to the name of the file.
filepath = 'data/Hydrology_ACGAA.csv'

# The King county files are not well formatted so we have to have a few extra lines here to clean
# the data
with open(filepath) as f:
    lines = [line.rstrip(',\n') for line in f]

# Write cleaned lines to a temporary in-memory string
cleaned_data = StringIO('\n'.join(lines))

# Now read with pandas
df = pd.read_csv(cleaned_data)

df

In [ ]:
# Download all of the data files from the the study area.
# Unfortunately, the provided files are not well formatted.
# The following code is necessary to correct the downloaded files. 
# The code also puts all of the data into one big output file.

data_dir = "data/"
df = king_county_csv_loader(data_dir)

In [ ]:
# Here's a simple bit of code to plot the time series
# from a single station

# Filter data
df_filtered = df[df.Site_Code == '43U']

# Plot
plt.plot(df_filtered['Collect Date (local)'], 
         df_filtered['Precipitation (inches)'])

plt.show()

In [ ]:
# This code makes a more nicely formatted plot.

fig, ax = plt.subplots(figsize=(10, 4))

# Filter data
df_filtered = df[df.Site_Code == '43U']

# Plot
ax.plot(df_filtered['Collect Date (local)'], 
        df_filtered['Precipitation (inches)'],
        marker='o', linestyle='-')

# Labels
ax.set_xlabel("Date")
ax.set_ylabel("Precipitation (inches)")
ax.grid(True)

# Format x-axis
locator = mdates.AutoDateLocator()
# formatter = mdates.ConciseDateFormatter(locator)
formatter = mdates.DateFormatter("%b %Y")  # e.g., 'Apr 2025'
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

fig.autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
# Assuming your DataFrame is called df
df['Collect Date (local)'] = pd.to_datetime(df['Collect Date (local)'])

# Define the hydrological year: assign the year of the following January to dates from Oct–Dec
df['Hydro Year'] = df['Collect Date (local)'].apply(
    lambda x: x.year + 1 if x.month >= 10 else x.year
)

# Group by Hydro Year and Site_Code and calculate average precipitation
avg_precip = df.groupby(['Site_Code', 'Hydro Year'])['Precipitation (inches)'].mean().reset_index()

# Rename for clarity
avg_precip.rename(columns={'Precipitation (inches)': 'Avg Precip (inches)'}, inplace=True)

# Display the new DataFrame
print(avg_precip)


In [ ]:
# Now examine just one extreme year
avg_precip[avg_precip['Hydro Year']==2019]

# Part 2: Theissen polygon ("Vornoi Diagram") maps

In [ ]:
# In this section, we plot the above data spatially.  We'll start by looking at 
# the 2019 hydrological year.

import geopandas as gpd
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.colors as mcolors

# Step 1: Load metadata and filter avg_precip
metadata = pd.read_csv('data/metadata.csv', comment='#')

# Filter avg_precip to hydrological year 2019
avg_2019 = avg_precip[avg_precip['Hydro Year'] == 2019]

# Merge on site code
merged_2019 = avg_2019.merge(metadata, left_on='Site_Code', right_on='Site_Code')

merged_2019


In [ ]:
# Next step: make an outline for the area of interest (AOI)
# do this by going to geojson.io, tracing out Vashon Island, and 
# then exporting the .geojson file and uploading it here.

geojson_path="data/map.geojson"
island_polygon = gpd.read_file(geojson_path).to_crs('EPSG:2926')
boundary_poly = island_polygon.unary_union
boundary_poly

In [ ]:
gdf_voronoi = make_voronoi_gdf(merged_2019, geojson_path=geojson_path)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
gdf_voronoi.plot(column='Avg Precip (inches)', cmap='viridis', edgecolor='k', legend=True,ax=ax)
plt.show()

In [ ]:
gdf_voronoi